In [3]:
import json
!pip install -U sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 3.8 MB/s eta 0:00:00


In [10]:
import pandas as pd

In [28]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 2: Load the datasets
job_requirements_file = 'new_job_descriptions.csv'

data = pd.read_csv('employee_data.csv')

# Specify the employee_id you want to filter by
employee = 178
person_data = data[data.iloc[:, 0] == employee]

# Load person skills dataset
person_interests = person_data['interest'].tolist()
person_skills = person_data['skills'].tolist()
person_cert = person_data['certification'].tolist()
person_jobhist = person_data['job history'].tolist()
person_psa = person_data['PSA courses attended'].tolist()

interests_embeddings = model.encode(person_interests)
skills_embeddings = model.encode(person_skills)
cert_embeddings=model.encode(person_cert)
jobhist_embeddings=model.encode(person_jobhist)
psa_embeddings=model.encode(person_psa)

# Load job requirements dataset
job_requirements_data = pd.read_csv(job_requirements_file)
job_titles = job_requirements_data['Job Title'].tolist()
job_descriptions = job_requirements_data['Job Description'].tolist()
job_requirements = job_requirements_data['Requirements'].tolist()

# Encode the job descriptions and job requirements for comparison
job_descriptions_embeddings = model.encode(job_descriptions)
job_requirements_embeddings = model.encode(job_requirements)

# Step 2: Combine the embeddings with specified weights
weighted_job_embeddings = 0.4 * job_descriptions_embeddings + 0.6 * job_requirements_embeddings
weighted_person_embeddings = 0.1 * interests_embeddings + 0.1 * psa_embeddings + 0.2 * skills_embeddings +0.3*cert_embeddings + 0.3*jobhist_embeddings

# Step 3: Calculate cosine similarity between weighted person embeddings and weighted job embeddings
similarity_matrix = cosine_similarity(weighted_person_embeddings, weighted_job_embeddings)

# Step 4: Find the most suitable job for the current employee
most_suitable_job_index = similarity_matrix.argmax(axis=1)[0]  # Get the index of the most suitable job

# Output the most suitable job for the current employee
print(f"Employee ID: {employee}")
print(f"Most Suitable Job: {job_titles[most_suitable_job_index]}")
print(f"Job Description: {job_descriptions[most_suitable_job_index]}")
print(f"Cosine Similarity Score: {similarity_matrix[0][most_suitable_job_index]:.4f}\n")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Employee ID: 178
Most Suitable Job: Senior Engineer / Engineer (System)
Job Description: You will be part of a dynamic team working on continuous improvement of PSA application platform-based systems. Key responsibilities include:

managing vendors for day-to-day system support
designing and developing system/mobile solutions to enhance business processes and digitalization objectives
engaging key stakeholders on process design and optimization, integration with PSA systems and industry partners
performing data support to engineers to ensure data quality and integrity for analysis and visualization
project management, vendor management and work collaboratively with key stakeholders to ensure quality delivery with corporate governance.
Cosine Similarity Score: 0.5342

